In [54]:
import importlib
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from imblearn.over_sampling import SMOTE

import Models.Reversible_Classifier as MC
import Models.Net as Net

import Data_Handler.Classifier_data as CD

Classifier_data = CD.Classifier_data
CNet = MC.Classifier

In [55]:
data_dir = 'Data'
g = pd.read_csv(os.path.join(data_dir, 
    'LupusGeneExpressionCompendium_AllGfeatures.csv'))
l = pd.read_csv(os.path.join(data_dir,
    'PatientDx_Labels.csv'))

df = pd.DataFrame({'trt':[0,1,2], 'sle_class':['Healthy_No_Treatment','SLE_No_Treatment', 'SLE_Treatment']})
#Merge the dataframe created with the labels dataset
df2 = pd.merge(l, df, how='outer', on='sle_class', left_on=None, right_on=None, left_index=False, right_index=False, sort=True, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

#Merge all datasets together
#research ready dataset
df3 = pd.merge(g, df2, how='inner', on='patid', left_on=None, right_on=None, left_index=False, right_index=False, sort=True, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

#split the dataset into data and labels

df_data = df3.iloc[:,1:-2]
df_labels = df3.loc[:, 'trt']

In [56]:
# Reload the imported libraries (optional)
importlib.reload(MC)
importlib.reload(Net)
importlib.reload(CD)

CNet = MC.Classifier
Classifier_data = CD.Classifier_data

# Call the Classifier_Data class with LupusGeneExpressionCompendium_AllGfeatures dataset
# Batch_size = 100
Lupus_data = Classifier_data(100, df_data.values, df_labels.values)

In [57]:
#Specify number of classes and features
N_outputs = Lupus_data.Num_of_classes
N_inputs = Lupus_data.Num_of_features
config = CNet.get_default_configs(N_inputs, N_outputs)
# config['hyperparameter']['lr'] = 3e-5
cnet = CNet(config, Lupus_data, [0.6, 0.2])
print(config)

Split with 945 members
Split with 315 members
Split with 316 members


/Users/roopal/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


{0: 782, 1: 782, 2: 782}


/Users/roopal/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


{0: 254, 1: 254, 2: 254}
{'random_seed': 123123123, 'result_dir': './results', 'data_dir': './Data', 'report_dir': './summaries', 'hyperparameters': {'Classifier': {'max_epochs': 30, 'lr': 0.0003, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08, 'l2': 0.0}, 'Generator': {'max_epochs': 30, 'lr': 0.0003, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08, 'alpha': 1.0}}, 'graph': {'layers': [15835, 512, 256, 128, 3], 'nonlinearities': ['lrelu', 'lrelu', 'lrelu', 'none'], 'G_layers': [128, 256, 512], 'G_nonlinearities': ['lrelu', 'lrelu', 'lrelu', 'relu'], 'depth': 1}}


In [58]:
# Train the classifier using optimized hyperparameter values
Hp_values = list([0.00228315,0.8978787,0.0020122,0.6406958,1.056737e-05])
Hp_names = ['lr', 'beta1', 'beta2', 'epsilon', 'max_epochs', 'l2_penalty']
Hp = dict(zip(Hp_names, Hp_values))
Hp['max_epochs'] = 100

cnet.train(save_every=0, eval_every=10, hyperparameters=Hp)

Initializing Variables with seed = 123123123
Starting Epoch 1 of 100
	Epoch Completed.  Training loss:   22.56437
Starting Epoch 2 of 100
	Epoch Completed.  Training loss:   64.79817
Starting Epoch 3 of 100
	Epoch Completed.  Training loss:   41.68647
Starting Epoch 4 of 100
	Epoch Completed.  Training loss:   37.89126
Starting Epoch 5 of 100
	Epoch Completed.  Training loss:   27.65114
Starting Epoch 6 of 100
	Epoch Completed.  Training loss:   26.10387
Starting Epoch 7 of 100
	Epoch Completed.  Training loss:   21.97616
Starting Epoch 8 of 100
	Epoch Completed.  Training loss:   21.07225
Starting Epoch 9 of 100
	Epoch Completed.  Training loss:   19.86786
Starting Epoch 10 of 100
	Epoch Completed.  Training loss:   18.98543
Evaluating model performance with validation data
	 Validation Loss:    5.45458
Confusion Matrix
	 [[ 51 121  82]
	 [ 25 183  46]
	 [  0   0 254]]
Starting Epoch 11 of 100
	Epoch Completed.  Training loss:   18.41162
Starting Epoch 12 of 100
	Epoch Completed.  Tra

	Epoch Completed.  Training loss:    1.89832
Starting Epoch 100 of 100
	Epoch Completed.  Training loss:    1.87556
Evaluating model performance with validation data
	 Validation Loss:    1.05567
Confusion Matrix
	 [[245   9   0]
	 [  6 240   8]
	 [  0   0 254]]
Evaluating model performance with validation data
	 Validation Loss:    1.05567
Confusion Matrix
	 [[245   9   0]
	 [  6 240   8]
	 [  0   0 254]]


In [59]:
### WARNING Generator Training is extremely resource intensive
# Train the generator using optimized hyperparameter values
Hp_values = list([0.278296,0.212388,0.340391,0.142364,58.039])
Hp_names = ['lr', 'beta1', 'beta2', 'epsilon', 'max_epochs', 'l2_penalty']
Hp = dict(zip(Hp_names, Hp_values))
Hp['max_epochs'] = 60
Hp['alpha'] = 0.05
print(Hp)

cnet.train_generator(save_every = 0, eval_every = 5, hyperparameters = Hp)

{'lr': 0.278296, 'beta1': 0.212388, 'beta2': 0.340391, 'epsilon': 0.142364, 'max_epochs': 60, 'alpha': 0.05}
starting Epoch 1 of 60
	Epoch Completed.  Training loss:    0.00192
Evaluating model performance with validation data
	 Validation Loss:    0.09152
Confusion Matrix
	 [[254   0   0]
	 [254   0   0]
	 [254   0   0]]
starting Epoch 2 of 60
	Epoch Completed.  Training loss:    0.00200
starting Epoch 3 of 60
	Epoch Completed.  Training loss:    0.00120
starting Epoch 4 of 60
	Epoch Completed.  Training loss:    0.00102
starting Epoch 5 of 60
	Epoch Completed.  Training loss:    0.00095
starting Epoch 6 of 60
	Epoch Completed.  Training loss:    0.00092
Evaluating model performance with validation data
	 Validation Loss:    0.00228
Confusion Matrix
	 [[244  10   0]
	 [  6 240   8]
	 [  0   0 254]]
starting Epoch 7 of 60
	Epoch Completed.  Training loss:    0.00090
starting Epoch 8 of 60
	Epoch Completed.  Training loss:    0.00088
starting Epoch 9 of 60
	Epoch Completed.  Training lo

In [60]:
# Get inverse from pseudoinverse
# Output = input-like matrix
Training_data = cnet.Training_data
batch = Training_data.getBatch(0)
batch_x = batch['X']
inverse_mat = cnet.sess.run(cnet.inverted_hidden_layer,
                           feed_dict = {cnet.input: batch_x})

print(inverse_mat.shape)

(100, 15835)


In [61]:
print(cnet.val_loss)

0.002049928017723279


In [62]:
cnet.sess.close()